In [2]:
import json
import requests
import urllib
from bs4 import BeautifulSoup
import os
import xml.etree.ElementTree as ET
import time
import pickle

In [3]:
def single_query(link, payload):
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print('WARNING', response.status_code)
    else:
        return response

In [4]:
getListURL = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
getListPayload = {}

In [5]:
allApps = single_query(getListURL, getListPayload)

In [6]:
allApps.text[0:100]

'{"applist":{"apps":[{"appid":403810,"name":"Molly"},{"appid":403830,"name":"The Consuming Shadow"},{'

In [7]:
allAppsJson = allApps.json()

In [8]:
allIds = allAppsJson['applist']['apps']

In [9]:
len(allIds)

94989

In [12]:
getAppIDUrl = 'http://store.steampowered.com/api/appdetails'

In [13]:
usecols = ['name', 'required_age', 'is_free', 'developers', 'publishers', 'platforms', 'genres', 'pc_requirements', 'release_date', 'price_overview']

a
b


In [23]:
Id0 = allIds[1]['appid']

In [24]:
print(Id0)

403830


In [25]:
resp =  single_query(getAppIDUrl, {'appids': Id0})

In [26]:
resp.json()[str(Id0)]['data']

{'type': 'game',
 'name': 'The Consuming Shadow',
 'steam_appid': 403830,
 'required_age': 0,
 'is_free': False,
 'controller_support': 'full',
 'detailed_description': "The Consuming Shadow is a procedural survival horror adventure in which you must explore the land, fight your way through randomly-generated dungeons and try to stay sane in your quest to save the world from the invading Ancients. The new Insanity Edition for Steam introduces new modes and features for even more expanded gameplay.<br />\r\n<br />\r\n- Roguelike perma-death gameplay<br />\r\n<br />\r\n- 4 playable characters to find and unlock, each with a different play style<br />\r\n<br />\r\n- 6 dungeon types, 10 mission types, and countless random encounters<br />\r\n<br />\r\n- 20 different monsters to discover and fight<br />\r\n<br />\r\n- Learn the 8 magic spells to gain the upper hand<br />\r\n<br />\r\n- Complete the bestiary and gather collectibles to uncover the world's lore<br />\r\n<br />\r\n- Insanity Ed

In [29]:
def parse_response_to_csv(query, usecols):
    result = []
    for col in usecols:
        result.append(query[col])
    return result.join(',')

In [28]:
def scrape_data(appList, usecols,  pickleFile):
    scraped = pickle.load(open(pickleFile, 'rb'))
    for app in appList:
        appId = app['appid']
        if appId not in scraped:
            time.sleep(1.6)
            query = single_query(getAppIDUrl, appId).json()
            if query[str(appId)]['data']['type'] == 'game':
                result = parse_response_to_csv(query[str(appId)]['data'], usecols)
            scraped.add(appId)
    pickle.dump(scraped, open(pickleFile, 'wb'))
    return result

In [30]:
def fullScrape(fullAppList, usecols, pickleFile, outputFile):
    output = open(outputFile, 'a')
    for i in range(0,len(fullAppList), 200):
        appList = fullAppList[i:i+200]
        result = scrape_date(appList, usecols, pickleFile)
        for r in result:
            output.write(','.join(r))

In [ ]:
fullScrape(allApps, usecols, 'scraped.p', 'raw_data.txt')